In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-4hhilxw6/unsloth_a6e3eb08ea9e427f8b7e7a88b62dac13
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-4hhilxw6/unsloth_a6e3eb08ea9e427f8b7e7a88b62dac13
  Resolved https://github.com/unslothai/unsloth.git to commit 4e570be9ae4ced8cdc64e498125708e34942befc
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 124.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 111.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 12.2 MB/s eta 0:00:00
   

In [ ]:
import torch
from unsloth import FastLanguageModel
from unsloth import to_sharegpt, standardize_sharegpt
from unsloth import is_bfloat16_supported
import json
from datasets import Dataset
import pandas as pd
import numpy as np
from trl import SFTTrainer
from transformers import TrainingArguments
from transformers import TextStreamer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [ ]:
MAX_LENGTH = 2048
DTYPE = None
load_in_4bit = True

In [ ]:


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = MAX_LENGTH,
    dtype = DTYPE,
    load_in_4bit = load_in_4bit,
    use_gradient_checkpointing = "unsloth",
)

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.43.3.
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

In [ ]:
peft_model = FastLanguageModel.get_peft_model(
              model = model,
              r = 16,
              target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                                "gate_proj", "up_proj", "down_proj",],
              lora_alpha = 16,
              lora_dropout = 0,
              bias = "none",
              use_gradient_checkpointing = "unsloth",
              random_state = 8080,
              use_rslora = False,
              loftq_config = None
          )

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


# Data

In [ ]:
DATA_PATH = "/content/drive/MyDrive/FinancialAdvisorProject/dataset_final.json"

In [ ]:
f = open(DATA_PATH)
data = f.read()

In [ ]:
finance_data = json.loads(data)

In [ ]:
finance_data[0]

{'about_me': 'I am a 21 year old college student.I was thinking of investing in the stock market.',
 'context': 'Meta fires 10k employees.\nMeta about to release Threads app.\nZuckerberg to visit China soon',
 'response': 'Monitor Meta due to layoffs and app release. Wait for stability before investing. Recent layoffs and impending app release may impact stock. Wait for stability.',
 'question': 'Is Meta a good stock to buy?'}

In [ ]:
merged_prompt = "[[ABOUT_ME:{about_me}]][[QUESTION:{question}]][[CONTEXT:{context}]]"

In [ ]:
finance_data = Dataset.from_pandas(pd.DataFrame(data=finance_data))

In [ ]:
dataset = to_sharegpt(
    dataset = finance_data,
    merged_prompt = merged_prompt,
    output_column_name = "response",
    conversation_extension = 5
)

Merging columns:   0%|          | 0/84 [00:00<?, ? examples/s]

Converting to ShareGPT:   0%|          | 0/84 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/84 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/84 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/84 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/84 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/84 [00:00<?, ? examples/s]

Extending conversations:   0%|          | 0/84 [00:00<?, ? examples/s]

In [ ]:
dataset = standardize_sharegpt(dataset = dataset)

Standardizing format:   0%|          | 0/84 [00:00<?, ? examples/s]

In [ ]:
dataset[2]

{'conversations': [{'content': 'ABOUT_ME:I am a 35 year old software engineer.I have been saving up to invest in real estate.QUESTION:Is now a good time to buy property?CONTEXT:Real estate market experiences surge in demand.\nInterest rates hit historic lows.\nGovernment announces new tax incentives for real estate investors.',
   'role': 'user'},
  {'content': 'Now is a good time for real estate investment. High demand, low rates, and tax incentives. Surge in demand, low rates, and tax incentives make it favorable for real estate investment.',
   'role': 'assistant'},
  {'content': 'ABOUT_ME:I am a 36 year old dentist.I want to invest in medical technology.QUESTION:Which healthcare companies are pioneering innovations in medical devices and diagnostics?CONTEXT:Medical technology companies revolutionize healthcare with advanced diagnostics, telemedicine, and robotic surgery.\nInvestors seek opportunities in personalized medicine, wearable devices, and digital health solutions.\nRegulat

In [3]:
merged_prompt = """
     You are a Financial Advisor and your main role is to be give suggestions. \
     User will tell about themselves first and ask you a question whether it is \
     good to invest in the stock. You will be given context as well which is the \
     latest news from Alpaca and based on the news and history, you need to \
     suggesting the user whether it is good to buy a stock or not. Along with \
     that, you will also provide the appropriate reasoning based on the context. \
     The input format will be ABOUT_ME:some text QUESTION:question CONTEXT:some text
     ABOUT_ME:{about_me} QUESTION:{question} CONTEXT:{context}
"""

In [ ]:
chat_template = """You are a Financial Advisor and your main role is to be give suggestions. \
     User will tell about themselves first and ask you a question whether it is \
     good to invest in the stock. You will be given context as well which is the \
     latest news from Alpaca and based on the news and history, you need to \
     suggesting the user whether it is good to buy a stock or not. Along with \
     that, you will also provide the appropriate reasoning based on the context. \
     The input format will be ABOUT_ME:some text QUESTION:question CONTEXT:some text
      >>> User details:
      {INPUT}
      >>> Your suggestion:
      {OUTPUT}"""

from unsloth import apply_chat_template
dataset = apply_chat_template(
    dataset,
    tokenizer = tokenizer,
    chat_template = chat_template,
    # default_system_message = "You are a helpful assistant", << [OPTIONAL]
)

Unsloth: We automatically added an EOS token to stop endless generations.


Map:   0%|          | 0/84 [00:00<?, ? examples/s]

# Training

In [ ]:
trainer = SFTTrainer(
    model = peft_model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    max_seq_length = MAX_LENGTH,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 50,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/84 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 84 | Num Epochs = 5
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 50
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.772500
2,1.741300
3,1.794400
4,1.782900
5,1.690800
6,1.610800
7,1.555400
8,1.300500
9,1.243500
10,1.157500


# Inference

In [4]:
merged_prompt = "ABOUT_ME:{}QUESTION:{}CONTEXT:{}"

In [ ]:
input_values = merged_prompt.format(
        "I am 30 yeard old and I am planning on a long term investment",
        "How is CDSL and can I invest in that?",
        "CDSL is a Central Depository where all the stocks when people invest go through that channel. Its a monopoly in India"
    )

In [ ]:
FastLanguageModel.for_inference(peft_model) # Enable native 2x faster inference
messages = [                    # Change below!
    {"role": "user", "content": merged_prompt.format(
        "I am 30 yeard old and I am planning on a long term investment",
        "Should I invest in CDSL?",
        "CDSL is a Central Depository where all the stocks when people invest go through that channel. Its a monopoly in India"
    )},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = peft_model.generate(input_ids, streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


 Invest in CDSL. It is a monopoly in the Indian Central Depository.<|end_of_text|>


In [ ]:
peft_model.save_pretrained('/content/drive/MyDrive/FinancialAdvisorProject/model')
tokenizer.save_pretrained('/content/drive/MyDrive/FinancialAdvisorProject/model')

('/content/drive/MyDrive/FinancialAdvisorProject/model/tokenizer_config.json',
 '/content/drive/MyDrive/FinancialAdvisorProject/model/special_tokens_map.json',
 '/content/drive/MyDrive/FinancialAdvisorProject/model/tokenizer.json')

In [ ]:
import os
os.environ['HF_TOKEN']="hf keys"

In [ ]:
hugging_face_modelname = "unsloth-finance-alpaca"
peft_model.push_to_hub(hugging_face_modelname, token = os.getenv('HF_TOKEN'))
tokenizer.push_to_hub(hugging_face_modelname, token = os.getenv('HF_TOKEN'))

README.md:   0%|          | 0.00/576 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Saved model to https://huggingface.co/unsloth-finance-alpaca


In [ ]:
peft_model.push_to_hub_merged(hugging_face_modelname,
                              tokenizer,
                              save_method='merged_16bit',
                              token = os.getenv('HF_TOKEN'))

Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 5.7G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 32.12 out of 52.96 RAM for saving.


100%|██████████| 32/32 [00:06<00:00,  4.82it/s]


Unsloth: Saving to organization with address mosrihari/unsloth-finance-alpaca
Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Unsloth: Saving to organization with address mosrihari/unsloth-finance-alpaca
Unsloth: Uploading all files... Please wait...


  0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Done.
Saved merged model to https://huggingface.co/None/unsloth-finance-alpaca


# Using saved model and upload to OLLAMA

# Push to Ollama

In [ ]:
import subprocess
import time

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


## convert the model to gguf

In [ ]:
peft_model.push_to_hub_gguf("mosrihari/unsloth-finance-alpaca",
                            tokenizer,
                            quantization_method = 'q4_k_m',
                            token = os.getenv('HF_TOKEN'))

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### Your chat template has a BOS token. We shall remove it temporarily.


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 34.83 out of 52.96 RAM for saving.


100%|██████████| 32/32 [00:03<00:00,  9.62it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


...

  0%|          | 0/1 [00:00<?, ?it/s]

unsloth.Q4_K_M.gguf:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Saved GGUF to https://huggingface.co/mosrihari/unsloth-finance-alpaca


Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### We removed it in GGUF's chat template for you.


Saved Ollama Modelfile to https://huggingface.co/mosrihari/unsloth-finance-alpaca


In [ ]:
subprocess.Popen(["ollama", "serve"])
time.sleep(3) # Wait for a few seconds for Ollama to load!

In [ ]:
print(tokenizer._ollama_modelfile)

FROM {__FILE_LOCATION__}

TEMPLATE """You are a Financial Advisor and your main role is to be give suggestions.      User will tell about themselves first and ask you a question whether it is      good to invest in the stock. You will be given context as well which is the      latest news from Alpaca and based on the news and history, you need to      suggesting the user whether it is good to buy a stock or not. Along with      that, you will also provide the appropriate reasoning based on the context.      The input format will be ABOUT_ME:some text QUESTION:question CONTEXT:some text{{ if .Prompt }}
      >>> User details:
      {{ .Prompt }}{{ end }}
      >>> Your suggestion:
      {{ .Response }}<|end_of_text|>"""

PARAMETER stop "<|start_header_id|>"
PARAMETER stop "<|end_header_id|>"
PARAMETER stop "<|eot_id|>"
PARAMETER stop "<|end_of_text|>"
PARAMETER stop "<|reserved_special_token_"


In [ ]:
#https://github.com/unslothai/unsloth/issues/748

In [ ]:
peft_model.save_pretrained_gguf("model",
                                tokenizer,
                                quantization_method = 'q4_k_m')

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 36.27 out of 52.96 RAM for saving.


100%|██████████| 32/32 [00:01<00:00, 16.19it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp will take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits will take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] will take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: [0] Installing llama.cpp. This will take 3 minutes...
Unsloth: [1] Converting model at model into bf16 GGUF format.
The output location will be ./model/unsloth.BF16.gguf
This will take 3 minutes...
INFO:hf-to-gguf:Loading model: model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Set model parameters
INFO:hf-to-gguf:gguf: context length = 8192
INFO:hf-to-gguf:gguf: embedding length = 4096
INFO:hf-to-gguf:gguf: feed forward length = 14336
INFO:hf-to-gguf:gguf: head count = 32
INFO:hf-to-gguf:gguf: